## load library yang akan digunakan

In [12]:
import pandas as dm
from openpyxl import load_workbook
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

## mengatur nama file excel dan sheet yang digunakan

In [13]:
wb = load_workbook(filename='D:\Studied_at_UNP\SEMESTER-5\praktikum_data_mining\praktikum_6\Data Alumni.xlsx')
sheet_ranges = wb['Sheet1']
df = dm.DataFrame(sheet_ranges.values)
df

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\asus\AppData\Local\Temp\ipykernel_26724\1141539022.py:1: SyntaxWarning: invalid escape sequence '\S'
  wb = load_workbook(filename='D:\Studied_at_UNP\SEMESTER-5\praktikum_data_mining\praktikum_6\Data Alumni.xlsx')


,0,1,2,3,4,5,6,7,8,9,10
0,No,NIM,TTL,IPK,Masa Tunggu,Tahun Masuk,Tahun Lulus,Gaji,TOEFL,Masa Studi,Umur
1,1,9018263,"Tapuih, 4 November 1989",2.13,2.4,2009,2016,2000000,390,7,34
2,2,9018269,"Koto Anau, 19 Desember 1990",2.75,2.4,2009,2016,2000000,413,7,33
3,3,9020026,"Lampung, 27 Maret 1992",3.24,2.4,2009,2016,2000000,400,7,31
4,4,12018060,"Padang, 23 Maret 1994",3.37,2.4,2012,2016,2000000,396,4,29
5,5,11018022,"Jakarta, 22 Juni 1993",3.2,2.4,2011,2016,2000000,460,5,30
6,6,9018212,"Pekanbaru, 3 Juni 1991",3.24,0,2009,2016,1200000,403,7,32
7,7,9018301,"Purbalingga, 08 Oktober 1991",3.06,2.4,2009,2016,2000000,450,7,32
8,8,12022020,"Bukittinggi, 29 Maret 1992",3.47,2.4,2012,2016,1500000,423,4,31


## seleksi kolom dari dataframe yang akan dilakukan forecasting

In [14]:
d = df[1:9][[1, 2, 3, 4, 5, 6, 7, 8]]
d.columns = ['NIM', 'TTL', 'IPK', 'Masa Tunggu', 'Tahun Masuk', 'Tahun Lulus', 'Gaji', 'TOEFL']
d

,NIM,TTL,IPK,Masa Tunggu,Tahun Masuk,Tahun Lulus,Gaji,TOEFL
1,9018263,"Tapuih, 4 November 1989",2.13,2.4,2009,2016,2000000,390
2,9018269,"Koto Anau, 19 Desember 1990",2.75,2.4,2009,2016,2000000,413
3,9020026,"Lampung, 27 Maret 1992",3.24,2.4,2009,2016,2000000,400
4,12018060,"Padang, 23 Maret 1994",3.37,2.4,2012,2016,2000000,396
5,11018022,"Jakarta, 22 Juni 1993",3.2,2.4,2011,2016,2000000,460
6,9018212,"Pekanbaru, 3 Juni 1991",3.24,0,2009,2016,1200000,403
7,9018301,"Purbalingga, 08 Oktober 1991",3.06,2.4,2009,2016,2000000,450
8,12022020,"Bukittinggi, 29 Maret 1992",3.47,2.4,2012,2016,1500000,423


## formatting data

In [17]:
d.dtypes
from datetime import datetime
now = datetime.now()

df.columns = df.iloc[0]
df = df.drop(0)

now.year
for index, row in df.iterrows():
    df.loc[index, 'Tahun Lahir'] = row['TTL'][-4:]
for index, row in df.iterrows():
    df.loc[index, 'Umur'] = now.year - int(row['Tahun Lahir'])
for index, row in df.iterrows():
    df.loc[index, 'MasaStudi'] = row['Tahun Lulus'] - row['Tahun Masuk']
df['IPK'] = df['IPK'].apply(str)
df['IPK'] = df['IPK'].str.replace(',', '.').apply(float)
df['Gaji'] = df['Gaji'].apply(float)
df


,No,NIM,TTL,IPK,Masa Tunggu,Tahun Masuk,Tahun Lulus,Gaji,TOEFL,Masa Studi,Umur,Tahun Lahir,MasaStudi
1,1,9018263,"Tapuih, 4 November 1989",2.13,2.4,2009,2016,2000000.0,390,7,36,1989,7.0
2,2,9018269,"Koto Anau, 19 Desember 1990",2.75,2.4,2009,2016,2000000.0,413,7,35,1990,7.0
3,3,9020026,"Lampung, 27 Maret 1992",3.24,2.4,2009,2016,2000000.0,400,7,33,1992,7.0
4,4,12018060,"Padang, 23 Maret 1994",3.37,2.4,2012,2016,2000000.0,396,4,31,1994,4.0
5,5,11018022,"Jakarta, 22 Juni 1993",3.20,2.4,2011,2016,2000000.0,460,5,32,1993,5.0
6,6,9018212,"Pekanbaru, 3 Juni 1991",3.24,0,2009,2016,1200000.0,403,7,34,1991,7.0
7,7,9018301,"Purbalingga, 08 Oktober 1991",3.06,2.4,2009,2016,2000000.0,450,7,34,1991,7.0
8,8,12022020,"Bukittinggi, 29 Maret 1992",3.47,2.4,2012,2016,1500000.0,423,4,33,1992,4.0


## ubah index dataframe

In [18]:
df_train = df[['NIM', 'IPK', 'Umur', 'MasaStudi', 'TOEFL', 'Masa Tunggu']].reset_index(drop=True)
df_train

for index, row in df_train.iterrows():
    if row['Masa Tunggu'] >= 2:
        df_train.loc[index, 'Lama Waktu Tunggu'] = 'Lama'
    elif row['Masa Tunggu'] >= 1:
        df_train.loc[index, 'Lama Waktu Tunggu'] = 'Sedang'
    else:
        df_train.loc[index, 'Lama Waktu Tunggu'] = 'Cepat'
df_train

,NIM,IPK,Umur,MasaStudi,TOEFL,Masa Tunggu,Lama Waktu Tunggu
0,9018263,2.13,36,7.0,390,2.4,Lama
1,9018269,2.75,35,7.0,413,2.4,Lama
2,9020026,3.24,33,7.0,400,2.4,Lama
3,12018060,3.37,31,4.0,396,2.4,Lama
4,11018022,3.20,32,5.0,460,2.4,Lama
5,9018212,3.24,34,7.0,403,0,Cepat
6,9018301,3.06,34,7.0,450,2.4,Lama
7,12022020,3.47,33,4.0,423,2.4,Lama


## menentukan kolom yang digunakan sebagai class targer forecasting

In [ ]:
y = df_train['Masa Tunggu'].values
y

array([2.4, 2.4, 2.4, 2.4, 2.4, 0, 2.4, 2.4], dtype=object)

## menentukan kolom apa saja yang digunakan sebagai fitur-fitur

In [ ]:
columns = ['IPK', 'TOEFL', 'MasaStudi', 'Umur']
features = df_train[list(columns)].values
features

array([[2.13, 390, 7.0, 36],
       [2.75, 413, 7.0, 35],
       [3.24, 400, 7.0, 33],
       [3.37, 396, 4.0, 31],
       [3.2, 460, 5.0, 32],
       [3.24, 403, 7.0, 34],
       [3.06, 450, 7.0, 34],
       [3.47, 423, 4.0, 33]], dtype=object)

## menyiapkan data fitur untuk proses training

In [ ]:

scaler = StandardScaler().fit(features)
x = scaler.transform(features)
x

array([[-2.29243943, -1.11533788,  0.75592895,  1.66666667],
       [-0.76002709, -0.16081616,  0.75592895,  1.        ],
       [ 0.45107299, -0.70032843,  0.75592895, -0.33333333],
       [ 0.77238525, -0.86633221, -1.51185789, -1.66666667],
       [ 0.35220767,  1.78972822, -0.75592895, -1.        ],
       [ 0.45107299, -0.5758256 ,  0.75592895,  0.33333333],
       [ 0.00617908,  1.37471878,  0.75592895,  0.33333333],
       [ 1.01954853,  0.25419328, -1.51185789, -0.33333333]])

## melakukan pelatihan menggunakan SVR scikit learn untuk membuat model forecasting 

In [ ]:

SVR_model = SVR(kernel='linear', C=100, gamma=.001).fit(x, y)
SVR_model

,kernel,'linear'
,degree,3
,gamma,0.001
,coef0,0.0
,tol,0.001
,C,100
,epsilon,0.1
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


# testing

## buat data uji dengan menggunakan dataframe yang sudah digunakan sebelumnya, ambil 3 data sercara acak untuk data uji.

In [ ]:
d_test = df_train.sample(n=3)
d_test

,NIM,IPK,Umur,MasaStudi,TOEFL,Masa Tunggu,Lama Waktu Tunggu
2,9020026,3.24,33,7.0,400,2.4,Lama
6,9018301,3.06,34,7.0,450,2.4,Lama
0,9018263,2.13,36,7.0,390,2.4,Lama


## menentukan kolom yang digunakan sebagai class target

In [ ]:
y_true = d_test['MasaStudi'].values
y_true

array([7., 7., 7.])

## menentukan kolom yang digunakan untuk fitur pada data uji

In [ ]:
features_test = d_test[list(columns)].values
features_test

array([[3.24, 400, 7.0, 33],
       [3.06, 450, 7.0, 34],
       [2.13, 390, 7.0, 36]], dtype=object)

## menyiapkan data fitur untuk proses testing

In [ ]:

scaler_test = StandardScaler().fit(features_test)
testdata = scaler_test.transform(features_test)
testdata

array([[ 0.88401795, -0.50800051,  0.        , -1.06904497],
       [ 0.51396393,  1.3970014 ,  0.        , -0.26726124],
       [-1.39798188, -0.88900089,  0.        ,  1.33630621]])

## melakukan prediksi menggunakan model klasifikasi atau classifier

In [ ]:
predict = SVR_model.predict(testdata)
predict

array([2.36479367, 2.34445755, 2.35303501])

## Man Absolute Deviation (MAD)

In [ ]:
mean_absolute_error(y_true, predict)

4.645904591784086